In [23]:
import tsl
import torch
import numpy as np
import pandas as pd
from tsl.datasets import MetrLA
from tsl.ops.connectivity import edge_index_to_adj
from tsl.data import SpatioTemporalDataset
from tsl.data.datamodule import (SpatioTemporalDataModule,
                                 TemporalSplitter)
from tsl.data.preprocessing import StandardScaler
import torch.nn as nn

from tsl.nn.blocks.encoders import RNN
from tsl.nn.layers import NodeEmbedding, DiffConv
from einops.layers.torch import Rearrange  # reshape data with Einstein notation

from tsl.metrics.torch import MaskedMAE, MaskedMAPE
from tsl.engines import Predictor

from pytorch_lightning.loggers import TensorBoardLogger
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from open_meteo import *
from tsl.data import Data

In [2]:
pin1 = '34.455737°S 70.925231°W'
pin2 = '34.452217°S 70.920077°W'
pin3 = '34.454563°S 70.911596°W'
pin4 = '34.457393°S 70.908903°W'
pin5 = '34.460729°S 70.914312°W'
pin6 = '34.456189°S 70.916805°W'
coords = [
    pin1, 
    pin2,
    pin3,
    pin4,
    pin5,
    pin6
]
# lat, lon = nsew_to_lat_long('34.455737°S 70.925231°W')
# # print(lat, long)
start_date = "2021-11-01"
end_date = "2024-03-01"  # 4 months later


# Get the weather data
w_list = get_weather_batch(coords,
                            start_d=start_date,
                            end_d=end_date)

distance_matrix = gen_distance_matrix(coords)

channels_df = format_result(w_list,
                            map_channel={
                                1: 'temperature_2m',
                                2: 'rel_hum_2m',
                                3: 'hourly_wind_speed_10m',
                                4: 'surf_pressure'
                            })

connectivity = distance_to_connectivity(distance_matrix)

In [24]:
channels_df

(20448, 24)

In [4]:
channels_df

Node                               1                                       2  \
Channel                            1     2          3           4          1   
date                                                                           
2021-11-01 00:00:00+00:00  16.650499  50.0   9.255571  986.389893  16.650499   
2021-11-01 01:00:00+00:00  15.400499  56.0   6.379216  986.453674  15.400499   
2021-11-01 02:00:00+00:00  14.700500  60.0   6.877790  986.574280  14.700500   
2021-11-01 03:00:00+00:00  13.700500  66.0   6.877790  986.371948  13.700500   
2021-11-01 04:00:00+00:00  13.500500  68.0   5.154416  986.059875  13.500500   
...                              ...   ...        ...         ...        ...   
2024-03-01 19:00:00+00:00  29.896999  31.0  16.179985  984.499756  29.896999   
2024-03-01 20:00:00+00:00  29.697001  32.0  17.551615  984.189514  29.697001   
2024-03-01 21:00:00+00:00  29.197001  35.0  16.781561  984.044983  29.197001   
2024-03-01 22:00:00+00:00  27.997000  41.0  15.281989  984.221863  27.997000   
2024-03-01 23:00:00+00:00  25.947001  47.0  10.895576  984.606567  25.947001   

Node                                                            3        ...  \
Channel                       2          3           4          1     2  ...   
date                                                                     ...   
2021-11-01 00:00:00+00:00  50.0   9.255571  986.389893  16.624500  50.0  ...   
2021-11-01 01:00:00+00:00  56.0   6.379216  986.453674  15.374499  56.0  ...   
2021-11-01 02:00:00+00:00  60.0   6.877790  986.574280  14.674500  60.0  ...   
2021-11-01 03:00:00+00:00  66.0   6.877790  986.371948  13.674500  66.0  ...   
2021-11-01 04:00:00+00:00  68.0   5.154416  986.059875  13.474500  68.0  ...   
...                         ...        ...         ...        ...   ...  ...   
2024-03-01 19:00:00+00:00  31.0  16.179985  984.499756  29.871000  31.0  ...   
2024-03-01 20:00:00+00:00  32.0  17.551615  984.189514  29.671001  32.0  ...   
2024-03-01 21:00:00+00:00  35.0  16.781561  984.044983  29.171001  35.0  ...   
2024-03-01 22:00:00+00:00  41.0  15.281989  984.221863  27.971001  41.0  ...   
2024-03-01 23:00:00+00:00  47.0  10.895576  984.606567  25.921001  47.0  ...   

Node                               4                      5                   \
Channel                            3           4          1     2          3   
date                                                                           
2021-11-01 00:00:00+00:00   9.255571  985.576172  16.611500  50.0   9.255571   
2021-11-01 01:00:00+00:00   6.379216  985.636414  15.361500  56.0   6.379216   
2021-11-01 02:00:00+00:00   6.877790  985.755066  14.661500  60.0   6.877790   
2021-11-01 03:00:00+00:00   6.877790  985.549683  13.661500  66.0   6.877790   
2021-11-01 04:00:00+00:00   5.154416  985.237305  13.461500  68.0   5.154416   
...                              ...         ...        ...   ...        ...   
2024-03-01 19:00:00+00:00  16.179985  983.723145  29.858000  31.0  16.179985   
2024-03-01 20:00:00+00:00  17.551615  983.412781  29.658001  32.0  17.551615   
2024-03-01 21:00:00+00:00  16.781561  983.267151  29.158001  35.0  16.781561   
2024-03-01 22:00:00+00:00  15.281989  983.440491  27.958000  41.0  15.281989   
2024-03-01 23:00:00+00:00  10.895576  983.819580  25.908001  47.0  10.895576   

Node                                           6                               
Channel                             4          1     2          3           4  
date                                                                           
2021-11-01 00:00:00+00:00  985.692688  16.631001  50.0   9.255571  986.041260  
2021-11-01 01:00:00+00:00  985.753235  15.381000  56.0   6.379216  986.103455  
2021-11-01 02:00:00+00:00  985.871948  14.681000  60.0   6.877790  986.223022  
2021-11-01 03:00:00+00:00  985.667175  13.681000  66.0   6.877790  986.019409  
2021-11-01 04:00:00+00:00  985.354736  13.481000  68.0   5.154416  985.707214

In [21]:
torch_dataset = SpatioTemporalDataset(target=channels_df,
                                      connectivity=connectivity,
                                      horizon=24,
                                      window=48,
                                      stride=1)



tensor([[ 16.6505,  50.0000,   9.2556, 986.3899],
        [ 16.6505,  50.0000,   9.2556, 986.3899],
        [ 16.6245,  50.0000,   9.2556, 985.9250],
        [ 16.6050,  50.0000,   9.2556, 985.5762],
        [ 16.6115,  50.0000,   9.2556, 985.6927],
        [ 16.6310,  50.0000,   9.2556, 986.0413]])

In [22]:
channels_df

Node                               1                                       2  \
Channel                            1     2          3           4          1   
date                                                                           
2021-11-01 00:00:00+00:00  16.650499  50.0   9.255571  986.389893  16.650499   
2021-11-01 01:00:00+00:00  15.400499  56.0   6.379216  986.453674  15.400499   
2021-11-01 02:00:00+00:00  14.700500  60.0   6.877790  986.574280  14.700500   
2021-11-01 03:00:00+00:00  13.700500  66.0   6.877790  986.371948  13.700500   
2021-11-01 04:00:00+00:00  13.500500  68.0   5.154416  986.059875  13.500500   
...                              ...   ...        ...         ...        ...   
2024-03-01 19:00:00+00:00  29.896999  31.0  16.179985  984.499756  29.896999   
2024-03-01 20:00:00+00:00  29.697001  32.0  17.551615  984.189514  29.697001   
2024-03-01 21:00:00+00:00  29.197001  35.0  16.781561  984.044983  29.197001   
2024-03-01 22:00:00+00:00  27.997000  41.0  15.281989  984.221863  27.997000   
2024-03-01 23:00:00+00:00  25.947001  47.0  10.895576  984.606567  25.947001   

Node                                                            3        ...  \
Channel                       2          3           4          1     2  ...   
date                                                                     ...   
2021-11-01 00:00:00+00:00  50.0   9.255571  986.389893  16.624500  50.0  ...   
2021-11-01 01:00:00+00:00  56.0   6.379216  986.453674  15.374499  56.0  ...   
2021-11-01 02:00:00+00:00  60.0   6.877790  986.574280  14.674500  60.0  ...   
2021-11-01 03:00:00+00:00  66.0   6.877790  986.371948  13.674500  66.0  ...   
2021-11-01 04:00:00+00:00  68.0   5.154416  986.059875  13.474500  68.0  ...   
...                         ...        ...         ...        ...   ...  ...   
2024-03-01 19:00:00+00:00  31.0  16.179985  984.499756  29.871000  31.0  ...   
2024-03-01 20:00:00+00:00  32.0  17.551615  984.189514  29.671001  32.0  ...   
2024-03-01 21:00:00+00:00  35.0  16.781561  984.044983  29.171001  35.0  ...   
2024-03-01 22:00:00+00:00  41.0  15.281989  984.221863  27.971001  41.0  ...   
2024-03-01 23:00:00+00:00  47.0  10.895576  984.606567  25.921001  47.0  ...   

Node                               4                      5                   \
Channel                            3           4          1     2          3   
date                                                                           
2021-11-01 00:00:00+00:00   9.255571  985.576172  16.611500  50.0   9.255571   
2021-11-01 01:00:00+00:00   6.379216  985.636414  15.361500  56.0   6.379216   
2021-11-01 02:00:00+00:00   6.877790  985.755066  14.661500  60.0   6.877790   
2021-11-01 03:00:00+00:00   6.877790  985.549683  13.661500  66.0   6.877790   
2021-11-01 04:00:00+00:00   5.154416  985.237305  13.461500  68.0   5.154416   
...                              ...         ...        ...   ...        ...   
2024-03-01 19:00:00+00:00  16.179985  983.723145  29.858000  31.0  16.179985   
2024-03-01 20:00:00+00:00  17.551615  983.412781  29.658001  32.0  17.551615   
2024-03-01 21:00:00+00:00  16.781561  983.267151  29.158001  35.0  16.781561   
2024-03-01 22:00:00+00:00  15.281989  983.440491  27.958000  41.0  15.281989   
2024-03-01 23:00:00+00:00  10.895576  983.819580  25.908001  47.0  10.895576   

Node                                           6                               
Channel                             4          1     2          3           4  
date                                                                           
2021-11-01 00:00:00+00:00  985.692688  16.631001  50.0   9.255571  986.041260  
2021-11-01 01:00:00+00:00  985.753235  15.381000  56.0   6.379216  986.103455  
2021-11-01 02:00:00+00:00  985.871948  14.681000  60.0   6.877790  986.223022  
2021-11-01 03:00:00+00:00  985.667175  13.681000  66.0   6.877790  986.019409  
2021-11-01 04:00:00+00:00  985.354736  13.481000  68.0   5.154416  985.707214